导包

In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
import pymysql

通过url爬取部门、日期、浏览量、正文

In [2]:
def parsecontent(newsurl):
#     处理空url
    if newsurl == '':
        url = "http://www.cuc.edu.cn/news/2021/0309/c1901a178590/page.htm"
    else:
        url = newsurl
    
    r = requests.get(url)
    rt = r.content
    rh = str(rt,"utf-8")
    soup = bs(rt,"html.parser")
#     print(soup)

#     h1 = soup.find_all("h1")
#     print(h1)

#     "class":"arti-name"中包含部门、日期、浏览量
    info = soup.find_all("span",attrs={"class":"arti-name"})
#     print(info)
    infotext = info[0].get_text()
#     print(infotext)

#     爬取部门
#     +3为了除去“来源：”占用的3个字符,截至到20，20为年份开头
    department = infotext[infotext.find('来源：')+3:infotext.find('20')]
#     print(department)

#     正则匹配
    rn = re.compile('[\u4e00-\u9fa5]+')
    dep = rn.findall(department)
#     print(dep[0])
    
#     爬取时间
    date = infotext[infotext.find('20'):infotext.find('20')+10]
#     print(date)

#     由于浏览量的位数不确定，故不采用此方法
#     count = infotext[infotext.find('浏览量：')+4:infotext.find('浏览量：')+6]
#     print(count)

#     爬取浏览量
    countinfo = soup.find_all("span",attrs={"class":"WP_VisitCount"})
    count = countinfo[0].get_text()
#     print(count)

#     爬取内容
    contentinfo = soup.find_all("div",attrs={"class":"wp_articlecontent"})
    content = contentinfo[0].get_text()
#     print(content)
    
    return dep[0],date,count,content

连接数据库并保存信息

In [3]:
def saveinfo(title,newsurl,department,date,count,content):
#     连接数据库
    db = pymysql.connect(host="localhost",user="zouzh",passwd="zouzihan0706",db="class")
    cursor = db.cursor()
#     cursor.execute("INSERT INTO cucnews(title,newsurl,department,date,count,content)VALUES(%s,%s,%s,%s,%s,%s)",(title,newsurl,department,date,count,content))
#     db.commit()
    try:
        cursor.execute("INSERT INTO cucnews(title,newsurl,department,date,count,content)VALUES(%s,%s,%s,%s,%s,%s)",(title,newsurl,department,date,count,content))
        db.commit()
        i = i+1
#         print("db-ok")
    except:
#         print("db-error")
        db.rollback()
    db.close()

主函数

In [4]:
urlbegin = "http://www.cuc.edu.cn/news/1901/list"
# k用于计数
k = 0
for i in range(1,581):
    url = urlbegin+str(i)+".htm"
    r = requests.get(url)
    rt = r.content
    rh = str(rt,"utf-8")
    soup = bs(rh,"html.parser")
    tit = soup.find_all("h3",attrs={'class','tit'})
    for t in tit:
        newsurl = t.find_all('a')
        urllen = str(newsurl[0]).find("target")
        title = t.get_text()
        newsurl = 'http://www.cuc.edu.cn' + (str(newsurl[0])[9:urllen-2])
#         对每个页面内容进行爬取
        department,date,count,content = parsecontent(newsurl)
#         测试
#         m = parsecontent(newsurl)
#         print(department)
        saveinfo(title,newsurl,department,date,count,content)
        k = k+1
        print(k)
#         print(title)
#         print(newsurl)
#         print(department)
#         print(date)
#         print(count)
#         print(content)
print("successful!")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
successful!
